In [60]:
%cd drive/MyDrive/Biocreative/

[Errno 2] No such file or directory: 'drive/MyDrive/Biocreative/'
/content/drive/MyDrive/Biocreative


In [27]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



### main model

In [28]:
!pip install transformers

In [29]:
!pip install pytorch-lightning

In [30]:
!pip install torchmetrics

In [31]:
import pandas as pd
import numpy as np

import random
from collections import defaultdict

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from torchmetrics.functional.classification.auroc import auroc

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline  
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

Global seed set to 42


42

In [32]:
df = pd.read_csv('Biocreative/preprocessed.csv')
df.head()

,abstract,Case Report,Diagnosis,Epidemic Forecasting,Mechanism,Prevention,Transmission,Treatment
0,December 2019 new highly contagious infectious...,0,0,0,1,0,0,1
1,"novel coronavirus disease COVID-19 , transmitt...",0,0,0,0,1,0,1
2,BACKGROUND December 2019 novel coronavirus SAR...,1,0,0,0,0,0,0
3,coronavirus disease 2019 COVID-19 pandemic imp...,0,0,0,0,1,0,0
4,OBJECTIVES Sofosbuvir daclatasvir direct-actin...,0,0,0,0,0,0,1


In [33]:
train_df, val_df = train_test_split(df, test_size=0.1)
train_df.shape, val_df.shape

((22464, 8), (2496, 8))

In [34]:
LABEL_COLUMNS = df.columns.tolist()[1:]

In [35]:
BERT_MODEL_NAME = 'dmis-lab/biobert-base-cased-v1.1'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)

----
### testing current approach 
----
*Note: below code is processed on single example*
- tokenization
- concationation of sequence and label encodings
- pass through model
- compute loss *(BCELoss)*

In [ ]:
sample_item = train_df.abstract[0]
print(sample_item)
encoding = tokenizer.encode_plus(
    sample_item,
    add_special_tokens=True,
    max_length=511,
    return_token_type_ids=False,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
)
encoding.keys()

In [ ]:
encoding["input_ids"].shape, encoding["attention_mask"].shape

In [ ]:
# encoding labels
sample = train_df.iloc[0]
label_dict = sample[1:].to_dict()
true_labels = [key for key, value in label_dict.items() if value == 1]
print(true_labels)
# see something for multiple labels

In [ ]:
label_encoding = tokenizer.encode_plus(
    true_labels[0],
    add_special_tokens=False,
    return_tensors="pt",
    return_token_type_ids=False
)
label_encoding.keys()

In [ ]:
label_encoding["input_ids"], tokenizer.convert_ids_to_tokens(label_encoding["input_ids"])

In [ ]:
abstract_input = encoding["input_ids"]
label_input = label_encoding["input_ids"]
input_tensor = torch.cat((abstract_input, label_input), 1)
print(f"abstract input encoding shape {abstract_input.shape}")
print(f"label input encoding shape {label_input.shape}")
print(f"resultant input encoding shape {input_tensor.shape}")

In [ ]:
# same goes with attention mask
attention_tensor = torch.cat((encoding["attention_mask"], label_encoding["attention_mask"]), 1)
print(attention_tensor.shape)

In [ ]:
# initialize bert model
bert_model = AutoModel.from_pretrained(BERT_MODEL_NAME)

In [ ]:
output = bert_model(input_tensor, attention_tensor)
print(output.last_hidden_state.shape, output.pooler_output.shape)

In [ ]:
# fully connected layer
fc_layer = nn.Linear(bert_model.config.hidden_size, 1)

In [ ]:
Output = fc_layer(output.pooler_output)
Output

In [ ]:
# define criterion
criterion = nn.BCELoss()

In [ ]:
truth_value = torch.Tensor([1])
truth_value.shape

In [ ]:
torch.sigmoid(Output)

In [ ]:
truth_value = torch.Tensor([1])

In [ ]:
loss = criterion(torch.sigmoid(Output), truth_value.view(1, 1))
loss

----
### uncomment below cells to create ZSL data
----
- *Note*: data has already been created @ /Biocreative/ZSL.csv

In [11]:
# Dataset Format 
# -----------------------------------
# |Sequence 1 | Annotation1 | Label |
# -----------------------------------
# |Sequence 2 | Annotation2 | Label |
# -----------------------------------

In [12]:
# def positive_data(example):

#   sequence = [example["abstract"]]
#   label_dict = example[1:].to_dict()
#   labels = [key for key, value in label_dict.items() if value == 1]
#   return zip(sequence * len(labels), labels)

In [13]:
# def negative_data(example):

#   sequence = [example["abstract"]]
#   label_dict = example[1:].to_dict()
#   labels = [key for key, value in label_dict.items() if value != 1]
#   label = labels[random.randint(0, len(labels)-1)]
#   return zip(sequence, [label])

In [14]:
# # conversion sequece
# train_dict = defaultdict(list)

# for idx, row in df.iterrows():
#   zipped_object = positive_data(row)
#   for key, value in zipped_object:
#     train_dict["Sequence"].append(key)
#     train_dict["Label"].append(value)
#     train_dict["Truth_Value"].append(True)
  
#   zipped_object = negative_data(row)
#   for key, value in zipped_object:
#     train_dict["Sequence"].append(key)
#     train_dict["Label"].append(value)
#     train_dict["Truth_Value"].append(False)    

In [15]:
# ZSL_df = pd.DataFrame.from_dict(train_dict, orient='index').T
# ZSL_df = ZSL_df.sample(frac=1).reset_index(drop=True)

In [ ]:
# ZSL_df.to_csv('drive/MyDrive/Biocreative/Biocreative/ZSL.csv', index=False)

In [ ]:
# data = pd.read_csv('Biocreative/ZSL.csv')
# data.rename(columns={
#     "Label": "Annotation",
#     "Truth_Value": "Label"
# }, inplace=True, errors='raise')
# data.head()

In [ ]:
# data.to_csv('Biocreative/ZSL.csv', index=False)

In [ ]:
# convert true/fa;se to 0/1
# data = pd.read_csv('Biocreative/ZSL.csv')
# data.Label = data.Label.astype(bool)
# data.head()
# data.to_csv('Biocreative/ZSL.csv', index=False)

### continuation

In [ ]:
#load ZSL data and start creating a model

In [36]:
data = pd.read_csv('Biocreative/ZSL.csv')
data.head()

,Sequence,Annotation,Label
0,PURPOSE analyse complication outcome COVID-19 ...,Mechanism,False
1,BACKGROUND Asymptomatic individual SARS-CoV-2 ...,Diagnosis,True
2,BACKGROUND COVID-19 critical pandemic affected...,Transmission,False
3,BACKGROUND Coronavirus disease 2019 COVID-19 p...,Epidemic Forecasting,False
4,safe effective vaccine candidate urgently need...,Mechanism,True


In [37]:
train_df, val_df = train_test_split(data, test_size=0.05)
train_df.shape, val_df.shape

((56240, 3), (2960, 3))

## data analysis


In [ ]:
pd.Series([(data.Label).sum(), (~data.Label).sum()], ["positive", "negative"]).sort_values().plot(kind="barh")

In [ ]:
token_counts = []

for _, row in data.iterrows():
  token_count = len(tokenizer.encode(
      row["Sequence"],
      max_length=512,
      truncation=True
  ))

  token_counts.append(token_count)

sns.histplot(token_counts)
plt.xlim([0, 512])

In [ ]:
# key observation: around 875 sentence are getting truncated and thus model will ignore the end context of those sequences.

----
## model classes 
----
- TopicAnnotationDataset
- TopicAnnotationDataLoader
- TopicAnnotationTagger

In [ ]:
# TopicAnnotationDataset
# TopicAnnotationDataLoader
# TopicAnnotationTagger

In [38]:
MAX_TOKEN_COUNT = 512

In [39]:
class TopicAnnotationDataset(Dataset):

  def __init__(
    self,
    data: pd.DataFrame,
    tokenizer: AutoTokenizer,
    max_token_len: int = 128
  ):
    self.tokenizer = tokenizer
    self.data = data
    self.max_token_len = max_token_len

  def __len__(self):
    return len(self.data)

  def concat_tensors(self, tensor1, tensor2):
    return torch.cat((tensor1, tensor2), 1)

  def __getitem__(self, index=int):
    
    data_row = self.data.iloc[index]

    sequence = data_row.Sequence
    annotation = data_row.Annotation
    label = data_row.Label.astype(int)

    sequence_encoding = self.tokenizer.encode_plus(
        sequence,
        add_special_tokens=True,
        max_length=self.max_token_len-3,
        return_token_type_ids=False,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    label_encoding = self.tokenizer.encode_plus(
        annotation,
        max_length=3,
        padding="max_length",
        add_special_tokens=False,
        return_token_type_ids=False,
        return_attention_mask=True,
        return_tensors="pt",
    )

    encoding = {
        "input_ids": self.concat_tensors(sequence_encoding["input_ids"], label_encoding["input_ids"]),
        "attention_mask": self.concat_tensors(sequence_encoding["attention_mask"], label_encoding["attention_mask"])
    }

    return dict(
        text=f"{sequence} {annotation}",
        input_ids=encoding["input_ids"].flatten(),
        attention_mask=encoding["attention_mask"].flatten(),
        label=torch.FloatTensor([label]) 
    )  

In [40]:
train_dataset = TopicAnnotationDataset(
    train_df,
    tokenizer,
    max_token_len=MAX_TOKEN_COUNT
)
sample_item = train_dataset[0]
sample_item.keys()

dict_keys(['text', 'input_ids', 'attention_mask', 'label'])

In [41]:
sample_item["input_ids"].shape, sample_item["attention_mask"].shape

(torch.Size([512]), torch.Size([512]))

----
### implementing a sample batch on bert model
----

In [ ]:
bert_model = AutoModel.from_pretrained(BERT_MODEL_NAME)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sample_batch = next(iter(DataLoader(train_dataset, batch_size=8, num_workers=2)))
sample_batch["input_ids"].shape, sample_batch["attention_mask"].shape

(torch.Size([8, 512]), torch.Size([8, 512]))

In [ ]:
output = bert_model(sample_batch["input_ids"], sample_batch["attention_mask"])

In [ ]:
output.last_hidden_state.shape, output.pooler_output.shape

(torch.Size([8, 512, 768]), torch.Size([8, 768]))

In [ ]:
output.pooler_output

tensor([[-0.1665,  0.0166,  0.9807,  ...,  0.9871, -0.2265,  0.9999],
        [-0.0495,  0.1043,  0.9562,  ...,  0.9703, -0.1793,  0.9998],
        [-0.1118,  0.1557,  0.9684,  ...,  0.9845, -0.2254,  1.0000],
        ...,
        [-0.0729,  0.0280,  0.9735,  ...,  0.9833, -0.2457,  0.9998],
        [-0.0466,  0.1762,  0.9508,  ...,  0.9637, -0.1084,  0.9997],
        [ 0.0209,  0.1466,  0.8724,  ...,  0.9168, -0.1825,  0.9995]],
       grad_fn=<TanhBackward>)

In [ ]:
# last hidden state shape
bert_model.config.hidden_size

768

### continutation

In [53]:
class TopicAnnotationDataModule(pl.LightningDataModule):

  def __init__(self, train_df, test_df, tokenizer, batch_size=8, max_token_len=128):
    
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.max_token_len = max_token_len

  def setup(self, stage=None):
    
    self.train_dataset =  TopicAnnotationDataset(
        self.train_df,
        self.tokenizer,
        self.max_token_len
    )

    self.test_dataset = TopicAnnotationDataset(
        self.test_df,
        self.tokenizer,
        self.max_token_len
    )

  def train_dataloader(self):

    return DataLoader(
      self.train_dataset,
      batch_size=self.batch_size,
      shuffle=True,
      num_workers=2
    )

  def val_dataloader(self):

    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=2
    )

  def test_dataloader(self):
    
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=2
    )

In [43]:
N_EPOCHS = 10
BATCH_SIZE = 8

data_module = TopicAnnotationDataModule(
    train_df,
    val_df,
    tokenizer,
    batch_size=BATCH_SIZE,
    max_token_len=MAX_TOKEN_COUNT
)

In [44]:
class TopicAnnotationTagger(pl.LightningModule):

  def __init__(
      self,
      n_classes: int,
      n_training_steps=None,
      n_warmup_steps=None
  ):

    super().__init__()
    self.bert = AutoModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
    self.classifier = nn.Linear(self.bert.config.hidden_size, 1)
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.criterion = nn.BCELoss()

  def forward(self, input_ids, attention_mask, label=None):

    output = self.bert(input_ids, attention_mask=attention_mask)
    output = self.classifier(output.pooler_output)
    output = torch.sigmoid(output)
    loss = 0
    if label is not None:
      loss = self.criterion(output, label)
    return loss, output

  def training_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    label = batch["label"]

    loss, outputs = self(input_ids, attention_mask, label)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions": outputs.detach(), "label": label}

  def validation_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    label = batch["label"]
    loss, outputs = self(input_ids, attention_mask, label)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    label = batch["label"]
    loss, outputs = self(input_ids, attention_mask, label)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def training_epoch_end(self, outputs):

    label = []
    predictions = []
    for output in outputs:
      for out_label in output["label"].detach().cpu():
        label.append(out_label)
      for out_predictions in output["predictions"].detach().cpu():
        predictions.append(out_predictions)

    label = torch.stack(label).int()
    predictions = torch.stack(predictions)

    roc_auc = auroc(predictions, label)
    self.logger.experiment.add_scalar("roc_auc/Train", roc_auc, self.current_epoch)
  
  def configure_optimizers(self):

    optimizer = AdamW(self.parameters(), lr=2e-5)

    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=self.n_warmup_steps,
      num_training_steps=self.n_training_steps
    )

    return dict(
        optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )

In [45]:
steps_per_epoch=len(train_df) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS

In [46]:
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

(14060, 70300)

In [47]:
model = TopicAnnotationTagger(
    n_classes=len(LABEL_COLUMNS),
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps
)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
checkpoint_callback = ModelCheckpoint(
    dirpath="ZSL_checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory ZSL_checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


In [49]:
logger = TensorBoardLogger("ZSL_lightning_logs", name="topic-annotations")

In [50]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

In [51]:
trainer = pl.Trainer(
    logger=logger,
    checkpoint_callback=True,
    callbacks=[checkpoint_callback, early_stopping_callback],
    max_epochs=N_EPOCHS,
    gpus=-1,
    progress_bar_refresh_rate=30
)   

In [25]:
trainer.fit(model, data_module)

In [ ]:
# train_at_epoch_end is functioning oddly so see through it by the end of the day

----
### testing
----

In [54]:
trained_model = TopicAnnotationTagger.load_from_checkpoint("/content/drive/MyDrive/Biocreative/ZSL_checkpoints/best-checkpoint.ckpt", n_classes=6)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [55]:
trained_model.freeze()

In [113]:
Sequence = data["Sequence"][3]
Annotation = data["Annotation"][3]

In [125]:
sequence_encoding = tokenizer.encode_plus(
    Sequence,
    add_special_tokens=True,
    max_length=MAX_TOKEN_COUNT-3,
    padding="max_length",
    return_attention_mask=True,
    return_token_type_ids=False,
    return_tensors="pt"
)
label_encoding = tokenizer.encode_plus(
    Annotation,
    max_length=3,
    add_special_tokens=False,
    padding="max_length",
    return_tensors="pt",
    return_token_type_ids=False
)
sequence_encoding.keys(), label_encoding.keys()

(dict_keys(['input_ids', 'attention_mask']),
 dict_keys(['input_ids', 'attention_mask']))

In [126]:
input_ids = torch.cat((sequence_encoding["input_ids"], label_encoding["input_ids"]), 1)
attention_mask = torch.cat((sequence_encoding["attention_mask"], label_encoding["attention_mask"]), 1)

In [127]:
_, test_predictions = trained_model(input_ids, attention_mask)
test_predictions

tensor([[0.0011]])

In [108]:
# checking on validation data
val_dict = val_df.iloc[19].to_dict()
print(val_dict)
val_Sequence = val_dict["Sequence"]
val_Annotation = val_dict["Annotation"]
val_Label = val_dict["Label"]
sequence_encoding = tokenizer.encode_plus(
    val_Sequence,
    add_special_tokens=True,
    max_length=MAX_TOKEN_COUNT-3,
    padding="max_length",
    return_attention_mask=True,
    return_token_type_ids=False,
    return_tensors="pt"
)
label_encoding = tokenizer.encode_plus(
    val_Annotation,
    max_length=3,
    add_special_tokens=False,
    padding="max_length",
    return_tensors="pt",
    return_token_type_ids=False
)
sequence_encoding.keys(), label_encoding.keys()

{'Sequence': 'OBJECTIVE report outcome patient multiple sclerosis MS related disorder coronavirus disease 2019 COVID-19 illness METHODS March 16 April 30 2020 patient MS related disorder NYU Langone MS Comprehensive Care Center identified laboratory-confirmed suspected COVID-19 diagnosis established using standardized questionnaire review in-patient hospital record RESULTS identified 76 patient 55 relapsing MS 9 pediatric onset 17 progressive MS 4 related disorder . Thirty-seven underwent PCR testing confirmed positive entire group 64 84 ) patient disease-modifying therapy DMT including anti-CD20 therapy n = 34 44.7 ) sphingosine-1-phosphate receptor modulators n = 10 13.5 ) common COVID-19 symptom fever cough 21.1 patient neurologic symptom recrudescence preceding coinciding infection total 18 23.7 ) hospitalized 8 10.5 ) COVID-19 critical illness related death Features common among hospitalized critical illness death older age presence comorbidities progressive disease nonambulatory 

(dict_keys(['input_ids', 'attention_mask']),
 dict_keys(['input_ids', 'attention_mask']))

In [109]:
input_ids = torch.cat((sequence_encoding["input_ids"], label_encoding["input_ids"]), 1)
attention_mask = torch.cat((sequence_encoding["attention_mask"], label_encoding["attention_mask"]), 1)

In [110]:
_, test_predictions = trained_model(input_ids, attention_mask)
test_predictions

tensor([[0.0032]])

In [112]:
train_df

,Sequence,Annotation,Label
33035,recent outbreak Coronavirus disease 2019 COVID...,Diagnosis,True
15997,OBJECTIVE Close contact novel coronavirus dise...,Prevention,True
45751,Concern coronavirus 2019 COVID-19 morbidity mo...,Mechanism,True
42614,SARS-CoV-2 RNA virus emerged December 2019 cit...,Mechanism,False
13438,first case coronavirus COVID-19 infection Arge...,Diagnosis,True
...,...,...,...
1969,COVID-19 major source fear stress anxiety well...,Mechanism,False
37488,Introduction Although medicine research curren...,Case Report,True
1872,Objective analyze clinical feature death-relat...,Transmission,False
35500,coronavirus disease 2019 COVID-19 pandemic wor...,Transmission,False
